In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\HAL 9004\\.conda\\envs\\spark-polito\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Excercise 31") \
    .getOrCreate()

In [3]:
spark

In [4]:
#Exercise 31 

# Log analysis
# Input: log of a web server (i.e., a textual file)
# Each line of the file is associated with a URL request

# Output: the list of distinct IP addresses associated
#with the connections to a google page (i.e.,
#connections to URLs containing the term
#“www.google.com”)

# Store the output in an HDFS folder 

## With RDD's

In [5]:
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=Excercise 31>

In [7]:
rdd_31 = sc.textFile("input_file_31.txt")

In [8]:
rdd_31.take(30)

['66.249.69.97 -- [24/Sep/2014:22:25:44 +0000] "GET http://www.google.com/bot.html”',
 '66.249.69.97 - - [24/Sep/2014:22:26:44 +0000] "GET http://www.google.com/how.html”',
 '66.249.69.97 - - [24/Sep/2014:22:28:44 +0000] "GET http://dbdmg.polito.it/course.html”',
 '71.19.157.179 - - [24/Sep/2014:22:30:12 +0000] "GET http://www.google.com/faq.html”',
 '66.249.69.95 -- [24/Sep/2014:31:28:44 +0000] "GET http://dbdmg.polito.it/thesis.html”',
 '66.249.69.97 -- [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”',
 '56.249.69.97 - - [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”']

In [9]:
rdd_31 = rdd_31.map(lambda a: a.split(" -")).map(lambda a: (a[0],a[1:]) ) 

In [10]:
rdd_31.take(30)

[('66.249.69.97',
  ['- [24/Sep/2014:22:25:44 +0000] "GET http://www.google.com/bot.html”']),
 ('66.249.69.97',
  ['', ' [24/Sep/2014:22:26:44 +0000] "GET http://www.google.com/how.html”']),
 ('66.249.69.97',
  ['',
   ' [24/Sep/2014:22:28:44 +0000] "GET http://dbdmg.polito.it/course.html”']),
 ('71.19.157.179',
  ['', ' [24/Sep/2014:22:30:12 +0000] "GET http://www.google.com/faq.html”']),
 ('66.249.69.95',
  ['- [24/Sep/2014:31:28:44 +0000] "GET http://dbdmg.polito.it/thesis.html”']),
 ('66.249.69.97',
  ['- [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”']),
 ('56.249.69.97',
  ['', ' [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”'])]

In [11]:
rdd_31 = rdd_31.countByKey()
rdd_31

defaultdict(int,
            {'66.249.69.97': 4,
             '71.19.157.179': 1,
             '66.249.69.95': 1,
             '56.249.69.97': 1})

In [12]:
rdd_31

defaultdict(int,
            {'66.249.69.97': 4,
             '71.19.157.179': 1,
             '66.249.69.95': 1,
             '56.249.69.97': 1})

In [13]:
for elem in rdd_31.keys():
    print (elem)

66.249.69.97
71.19.157.179
66.249.69.95
56.249.69.97


## With DataFrames

In [62]:
df31 = spark.read.csv("input_file_31.txt")

In [63]:
df31

DataFrame[_c0: string]

In [64]:
df31.take(10)

[Row(_c0='66.249.69.97 -- [24/Sep/2014:22:25:44 +0000] "GET http://www.google.com/bot.html”'),
 Row(_c0='66.249.69.97 - - [24/Sep/2014:22:26:44 +0000] "GET http://www.google.com/how.html”'),
 Row(_c0='66.249.69.97 - - [24/Sep/2014:22:28:44 +0000] "GET http://dbdmg.polito.it/course.html”'),
 Row(_c0='71.19.157.179 - - [24/Sep/2014:22:30:12 +0000] "GET http://www.google.com/faq.html”'),
 Row(_c0='66.249.69.95 -- [24/Sep/2014:31:28:44 +0000] "GET http://dbdmg.polito.it/thesis.html”'),
 Row(_c0='66.249.69.97 -- [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”'),
 Row(_c0='56.249.69.97 - - [24/Sep/2014:56:26:44 +0000] "GET http://www.google.com/how.html”')]

In [65]:
df31.select("*").show()

+--------------------+
|                 _c0|
+--------------------+
|66.249.69.97 -- [...|
|66.249.69.97 - - ...|
|66.249.69.97 - - ...|
|71.19.157.179 - -...|
|66.249.69.95 -- [...|
|66.249.69.97 -- [...|
|56.249.69.97 - - ...|
+--------------------+



In [66]:
from pyspark.sql.functions import split,col

df31 = df31.select(split(col("_c0")," -")[0].alias("Name_Array"))

In [67]:
df31.take(10)

[Row(Name_Array='66.249.69.97'),
 Row(Name_Array='66.249.69.97'),
 Row(Name_Array='66.249.69.97'),
 Row(Name_Array='71.19.157.179'),
 Row(Name_Array='66.249.69.95'),
 Row(Name_Array='66.249.69.97'),
 Row(Name_Array='56.249.69.97')]

In [69]:
df31.select("*").distinct().show()

+-------------+
|   Name_Array|
+-------------+
|71.19.157.179|
| 66.249.69.97|
| 56.249.69.97|
| 66.249.69.95|
+-------------+



In [ ]:
df30.write.format("csv").save("spark_output/datacsv")